# core

> Fill in a module description here

In [1]:
#| default_exp core

In [2]:
import importlib
import pkgutil
from astroid import MANAGER, FunctionDef, ClassDef
import logging
import argparse
import traceback

In [50]:
#|hide
from fastcore.test import test_eq
from astroid import parse, extract_node

In [42]:
#| export
def format_symbol(name, signature, doc, decorators=None, is_method=False):
    "format the information in markdown"
    params = signature.split('(', 1)[1].rsplit(')', 1)[0] if '(' in signature else ''
    decorator_str = ' '.join(f'@{d}' for d in decorators) + ' ' if decorators else ''
    formatted = f"- `{decorator_str}{name}({params})`\n" if is_method else f"- `{decorator_str}def {name}({params})`\n"    
    if doc:
        doc_lines = doc.strip().split('\n')
        formatted += '    ' + '\n    '.join(doc_lines) + '\n'
    return formatted

In [43]:
#|hide
test_eq(format_symbol("hello", "hello()", "This is a test function"), "- `def hello()`\n    This is a test function\n")

# Test method formatting
test_eq(format_symbol("world", "world(self)", "A method", is_method=True), "- `world(self)`\n    A method\n")

# Test with decorators
test_eq(format_symbol("decorated", "decorated()", "Decorated function", decorators=["decorator1", "decorator2"]), "- `@decorator1 @decorator2 def decorated()`\n    Decorated function\n")

# Test with parameters
test_eq(format_symbol("params", "params(a, b, c=None)", "Function with parameters"), "- `def params(a, b, c=None)`\n    Function with parameters\n")

# Test with multi-line docstring
test_eq(format_symbol("multiline", "multiline()", "First line\nSecond line\nThird line"), "- `def multiline()`\n    First line\n    Second line\n    Third line\n")

# Test with no docstring
test_eq(format_symbol("no_doc", "no_doc()", ""), "- `def no_doc()`\n")

# Test method with decorators and parameters
test_eq(format_symbol("complex", "complex(self, x, y=0)", "Complex method", decorators=["classmethod"], is_method=True), "- `@classmethod complex(self, x, y=0)`\n    Complex method\n")

In [44]:
#| export
def is_public_symbol(name): return not name.startswith('_') or (name.startswith('__') and name.endswith('__'))
def is_valid_method(method, method_name): return isinstance(method, FunctionDef) and is_public_symbol(method_name)
def get_params(func): return ', '.join(arg.name for arg in func.args.args)
def get_decorators(obj): return [d.as_string() for d in obj.decorators.nodes] if obj.decorators else []
def log_error(name, error): raise RuntimeError(f"Error processing symbol {name}: {str(error)}")

In [47]:
def process_function(func, name, include_no_docstring):
    "Parse functions"
    params = get_params(func)
    signature = f"{name}({params})"
    doc = func.doc_node.value if func.doc_node else ""
    decorators = get_decorators(func)
    if include_no_docstring or doc:
        return ('function', name, signature, doc, decorators)
    return None

In [46]:
#|hide
# Test case 1: Function with docstring
func_with_doc = parse('''
def test_func(a, b):
    """This is a test function"""
    pass
''').body[0]

result = process_function(func_with_doc, 'test_func', False)
test_eq(result, ('function', 'test_func', 'test_func(a, b)', 'This is a test function', []))

In [66]:
#|export
def _process_method(method, method_name):
    method_params = get_params(method)
    method_signature = f"{method_name}({method_params})"
    method_doc = method.doc_node.value if method.doc_node else ""
    method_decorators = get_decorators(method)
    return (method_name, method_signature, method_doc, method_decorators)

def process_class(cls, name, include_no_docstring):
    "Parse classes."
    class_doc = cls.doc_node.value if cls.doc_node else ""
    class_decorators = get_decorators(cls)
    methods = [_process_method(method, method_name) 
               for method_name, method in cls.items() 
               if is_valid_method(method, method_name)]
    return ('class', name, class_doc, class_decorators, methods)

In [67]:
#|hide
mock_class = extract_node('''
@decorator
class TestClass:
    """
    Class docstring
    with multiple lines
    """
    @staticmethod
    def method1(arg1):
        """
        Method1 docstring
        with multiple lines
        """
        pass
    def method2(self):
        """Single line docstring"""
        pass
''')

# Process the class
result = process_class(mock_class, "TestClass", True)

# Test equality
test_eq(result, (
    'class',
    'TestClass',
    '\n    Class docstring\n    with multiple lines\n    ',
    ['decorator'],
    [
        ('method1', 'method1(arg1)', '\n        Method1 docstring\n        with multiple lines\n        ', ['staticmethod']),
        ('method2', 'method2(self)', 'Single line docstring', [])
    ]
))

In [68]:
def get_public_symbols(module, include_no_docstring):
    symbols = []
    for name, obj in module.items():
        if is_public_symbol(name):
            try:
                if isinstance(obj, FunctionDef):
                    symbol = process_function(obj, name, include_no_docstring)
                    if symbol: symbols.append(symbol)
                elif isinstance(obj, ClassDef):
                    symbols.append(process_class(obj, name, include_no_docstring))
            except Exception as e: log_error(name, e)
    return symbols

In [77]:
#|hide
test_module = parse('''
def public_function():
    """This is a public function"""
    pass

def _private_function(): pass

class PublicClass:
    """This is a public class"""
    def method(self):
        pass
    def _private_method(): pass

_private_function = lambda: None
''')

# Test 1: Get public symbols without including symbols that lack docstrings
result = get_public_symbols(test_module, False)

# Should have 2 public symbols
test_eq(len(result), 2)
# First item should be public_function
test_eq(result[0][1], 'public_function')
# Second item should be PublicClass
test_eq(result[1][1], 'PublicClass')

# Test 2: Get public symbols including symbols that lack docstrings
result_with_no_docstring = get_public_symbols(test_module, True)

# Should still have 2 public symbols
test_eq(len(result_with_no_docstring), 2)
# PublicClass should have 1 public method
test_eq(len(result_with_no_docstring[1][4]), 1)
# PublicClass should include 'method'
test_eq(result_with_no_docstring[1][4][0][0], 'method')


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()